# Seq2Seq 模型介绍

Seq2Seq（Sequence to Sequence）是一种用于处理序列数据的深度学习模型，广泛应用于自然语言处理（NLP）任务中，如机器翻译、文本摘要、对话生成等。Seq2Seq模型由两个主要部分组成：编码器（Encoder）和解码器（Decoder）。

## 1. 编码器（Encoder）

编码器的作用是将输入序列（如一句话）转换为一个固定大小的向量，称为上下文向量（Context Vector）或编码状态（Encoded State）。这个向量包含了输入序列的所有信息，可以被解码器用来生成输出序列。

- **输入**：一个序列（如一句话）。
- **输出**：一个固定大小的向量（上下文向量）。

编码器通常使用循环神经网络（RNN）、长短期记忆网络（LSTM）或门控循环单元（GRU）来处理输入序列。每个时间步的输入会被编码器处理，并逐步更新其内部状态，直到整个序列被处理完毕。最终的内部状态就是上下文向量。

## 2. 解码器（Decoder）

解码器的作用是根据编码器生成的上下文向量，逐步生成输出序列（如翻译后的句子）。解码器通常也是一个RNN、LSTM或GRU，但它从上下文向量开始，逐步生成输出序列的每个元素。

- **输入**：上下文向量。
- **输出**：一个序列（如翻译后的句子）。

在生成输出序列时，解码器通常会使用一种叫做“教师强制”（Teacher Forcing）的技术，即在训练过程中，解码器的输入是目标序列的前一个元素，而不是解码器自己生成的上一个元素。这有助于模型更快地收敛。

## 3. 注意力机制（Attention Mechanism）

原始的Seq2Seq模型存在一个问题：编码器生成的上下文向量需要包含输入序列的所有信息，这可能导致信息压缩和丢失。为了解决这个问题，注意力机制被引入到Seq2Seq模型中。

注意力机制允许解码器在生成每个输出元素时，关注输入序列的不同部分。具体来说，解码器在每个时间步会计算一个注意力权重向量，该向量表示输入序列中每个元素对当前输出元素的重要性。然后，解码器根据这些权重对输入序列进行加权求和，得到一个加权上下文向量，用于生成当前输出元素。

## 工作流程
1. 输入处理：将输入序列（如一个句子）分解成单词或字符，并转换为向量表示。
2. 编码：编码器接收输入序列，逐步处理每个元素，最终生成一个上下文向量，这个向量包含了输入序列的信息。
3. 解码：解码器以上下文向量为初始输入，逐步生成输出序列，通常使用“教师强制”（Teacher Forcing）策略，即在每个时间步使用真实的上一个输出作为下一个输入。
4. 损失计算：通过计算生成的序列与真实序列之间的差异（如使用交叉熵损失）来训练模型。

![S2S](https://zh-v2.d2l.ai/_images/seq2seq-details.svg "s2s")

## 4. 应用场景

- **机器翻译**：将一种语言的句子翻译成另一种语言。
- **文本摘要**：将长文本压缩成简短的摘要。
- **对话生成**：根据用户的输入生成合适的回复。
- **语音识别**：将语音信号转换为文本。

## 5. 优点与局限性

- **优点**：
  - 能够处理变长的输入和输出序列。
  - 通过注意力机制，可以更好地捕捉输入序列中的重要信息。

- **局限性**：
  - 训练时间较长，尤其是在处理长序列时。
  - 对于非常长的序列，信息压缩可能导致性能下降。

## 简单代码实现

### 编码器


In [ ]:
#@save
class Seq2SeqEncoder(d2l.Encoder):
    """用于序列到序列学习的循环神经网络编码器"""
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                 dropout=0, **kwargs):
        super(Seq2SeqEncoder, self).__init__(**kwargs)
        # 嵌入层
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.GRU(embed_size, num_hiddens, num_layers,
                          dropout=dropout)

    def forward(self, X, *args):
        # 输出'X'的形状：(batch_size,num_steps,embed_size)
        X = self.embedding(X)
        # 在循环神经网络模型中，第一个轴对应于时间步
        X = X.permute(1, 0, 2)
        # 如果未提及状态，则默认为0
        output, state = self.rnn(X)
        # output的形状:(num_steps,batch_size,num_hiddens)
        # state的形状:(num_layers,batch_size,num_hiddens)
        return output, state

### 解码器

In [ ]:
class Seq2SeqDecoder(d2l.Decoder):
    """用于序列到序列学习的循环神经网络解码器"""
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                 dropout=0, **kwargs):
        super(Seq2SeqDecoder, self).__init__(**kwargs)
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.GRU(embed_size + num_hiddens, num_hiddens, num_layers,
                          dropout=dropout)
        self.dense = nn.Linear(num_hiddens, vocab_size)

    def init_state(self, enc_outputs, *args):
        return enc_outputs[1]

    def forward(self, X, state):
        # 输出'X'的形状：(batch_size,num_steps,embed_size)
        X = self.embedding(X).permute(1, 0, 2)
        # 广播context，使其具有与X相同的num_steps
        context = state[-1].repeat(X.shape[0], 1, 1)
        X_and_context = torch.cat((X, context), 2)
        output, state = self.rnn(X_and_context, state)
        output = self.dense(output).permute(1, 0, 2)
        # output的形状:(batch_size,num_steps,vocab_size)
        # state的形状:(num_layers,batch_size,num_hiddens)
        return output, state

### 损失函数

In [ ]:
#@save
def sequence_mask(X, valid_len, value=0):
    """在序列中屏蔽不相关的项"""
    maxlen = X.size(1)
    mask = torch.arange((maxlen), dtype=torch.float32,
                        device=X.device)[None, :] < valid_len[:, None]
    X[~mask] = value
    return X

X = torch.tensor([[1, 2, 3], [4, 5, 6]])
sequence_mask(X, torch.tensor([1, 2]))

In [ ]:
#@save
class MaskedSoftmaxCELoss(nn.CrossEntropyLoss):
    """带遮蔽的softmax交叉熵损失函数"""
    # pred的形状：(batch_size,num_steps,vocab_size)
    # label的形状：(batch_size,num_steps)
    # valid_len的形状：(batch_size,)
    def forward(self, pred, label, valid_len):
        weights = torch.ones_like(label)
        weights = sequence_mask(weights, valid_len)
        self.reduction='none'
        unweighted_loss = super(MaskedSoftmaxCELoss, self).forward(
            pred.permute(0, 2, 1), label)
        weighted_loss = (unweighted_loss * weights).mean(dim=1)
        return weighted_loss